In [9]:
!pip install wandb
!pip install tensorflow-gpu

In [0]:
# Import layers
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
import pandas as pd
import numpy as np
import cv2 as cv2
import keras
import subprocess
import os
import time

In [11]:

import wandb
from wandb.keras import WandbCallback

run = wandb.init()
config = run.config

# set hyperparameters
batch_size = 32
num_epochs = 5

input_shape = (48, 48, 1)



In [0]:
class Perf(Callback):
    """Performance callback for logging inference time"""

    def __init__(self, testX):
        self.testX = testX

    def on_epoch_end(self, epoch, logs):
        start = time.time()
        self.model.predict(self.testX)
        end = time.time()
        self.model.predict(self.testX[:1])
        latency = time.time() - end
        wandb.log({"avg_inference_time": (end - start) /
                   len(self.testX) * 1000, "latency": latency * 1000}, commit=False)


def load_fer2013():
    """Load the emotion dataset"""
    if not os.path.exists("fer2013"):
        print("Downloading the face emotion dataset...")
        subprocess.check_output(
            "curl -SL https://www.dropbox.com/s/opuvvdv3uligypx/fer2013.tar | tar xz", shell=True)
    print("Loading dataset...")
    data = pd.read_csv("fer2013/fer2013.csv")
    pixels = data['pixels'].tolist()
    width, height = 48, 48
    faces = []
    for pixel_sequence in pixels:
        face = np.asarray(pixel_sequence.split(' '), dtype=np.uint8).reshape(width, height)
        face = cv2.resize(face.astype('uint8'), (width, height))
        faces.append(face.astype('float32'))

    faces = np.asarray(faces)
    faces = np.expand_dims(faces, -1)
    emotions = pd.get_dummies(data['emotion']).as_matrix()

    val_faces = faces[int(len(faces) * 0.8):]
    val_emotions = emotions[int(len(faces) * 0.8):]
    train_faces = faces[:int(len(faces) * 0.8)]
    train_emotions = emotions[:int(len(faces) * 0.8)]

    return train_faces, train_emotions, val_faces, val_emotions



In [20]:
# loading dataset
train_faces, train_emotions, val_faces, val_emotions = load_fer2013()
num_samples, num_classes = train_emotions.shape

train_faces /= 255.
val_faces /= 255.

Loading dataset...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [23]:

# Define the model here, CHANGEME
model = Sequential()
model.add(Conv2D(32, (3,3), activation="relu", input_shape=input_shape))
model.add(Dropout(.25))
model.add(MaxPooling2D())
model.add(Conv2D(64, (3,3), activation="relu"))
model.add(Dropout(.3))
model.add(MaxPooling2D())
model.add(Conv2D(128, (3,3), activation="relu"))
model.add(Dropout(.3))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(1024, activation="relu"))
model.add(Dropout(.3))
model.add(Dense(64, activation="relu"))
model.add(Dropout(.3))
model.add(Dense(num_classes, activation="softmax"))
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

# log the number of total parameters
# config.total_params = model.count_params()
model.fit(train_faces, train_emotions, batch_size=64,
          epochs=30, verbose=1, callbacks=[
              Perf(val_faces),
              WandbCallback(data_type="image", labels=[
                            "Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"])
          ], validation_data=(val_faces, val_emotions))

# save the model
model.save("emotion.h5")

Train on 28709 samples, validate on 7178 samples
Epoch 1/30
28608/28709 [============================>.] - ETA: 0s - loss: 1.7233 - accuracy: 0.3026

wandb: WARNING No validation_data set, pass a generator to the callback.


28709/28709 [==============================] - 12s 428us/sample - loss: 1.7228 - accuracy: 0.3028 - val_loss: 1.5879 - val_accuracy: 0.3670
Epoch 2/30
28672/28709 [============================>.] - ETA: 0s - loss: 1.4718 - accuracy: 0.4320

wandb: WARNING No validation_data set, pass a generator to the callback.


28709/28709 [==============================] - 10s 349us/sample - loss: 1.4718 - accuracy: 0.4321 - val_loss: 1.4312 - val_accuracy: 0.4719
Epoch 3/30
28672/28709 [============================>.] - ETA: 0s - loss: 1.3665 - accuracy: 0.4804

...

28709/28709 [==============================] - 10s 345us/sample - loss: 0.4849 - accuracy: 0.8263 - val_loss: 1.1422 - val_accuracy: 0.6088
Epoch 29/30
28672/28709 [============================>.] - ETA: 0s - loss: 0.4732 - accuracy: 0.8323

wandb: WARNING No validation_data set, pass a generator to the callback.


28709/28709 [==============================] - 10s 346us/sample - loss: 0.4732 - accuracy: 0.8322 - val_loss: 1.1697 - val_accuracy: 0.6078
Epoch 30/30
28672/28709 [============================>.] - ETA: 0s - loss: 0.4523 - accuracy: 0.8390

wandb: WARNING No validation_data set, pass a generator to the callback.


28709/28709 [==============================] - 10s 348us/sample - loss: 0.4523 - accuracy: 0.8390 - val_loss: 1.1803 - val_accuracy: 0.6098


In [0]:
import subprocess
shellscript = subprocess.Popen(["shellscript.sh"], stdin=subprocess.PIPE)
